루브릭
1. 기존 데이터셋을 추가 정제하고, generation 성능을 끌어올리기 위한 기법들을 실험해 모델 perfomance를 향상시켜보았는가?	
- 기존 데이터셋의 문제점을 분석하고 전처리 전략을 수립해 추가 정제를 진행했다. Beam search, Top-k(p) sampling 등 최선의 디코딩 전략을 수립해 향상된 모델 추론 결과를 제시했다. BLEU, ROUGE 등 생성된 텍스트를 평가하기 위한 메트릭을 적용한 정량적인 평가 결과와 주관적인 평가를 비교분석하였다.

2. 새로운 데이터를 수집해 전처리를 수행하여 모델을 재학습시켜보았는가?

- 모두의 말뭉치, AI hub 등에 공개된 데이터를 사용해 추가 데이터셋을 구축하기 위한 기준과 근거를 수립했다. ChatGPT API나 다양한 한국어 benchmark 데이터셋을 활용해 Human Feedback 을 대체할 수 있는 아이디어를 구현했다. 위를 바탕으로 SFT, RM, PPO 세 단계에 필요한 각 데이터셋을 적절히 구축하여, 모델 추론 결과와 수립한 가설을 비교해보았다.


3. 학습 전략 또는 foundation model을 변경해 모델을 재학습시켜보았는가?	
- 더 적절한 Instruction Tuning 기법을 적용해 SFT를 해보거나, Reward Model의 ranking algorithm을 개선해보았다. KoGPT-2가 아닌 다른 모델을 initial model로 사용하여 모델 학습을 성공시켰다. 허깅페이스의 accelerate, bitsandbytes 라이브러리 등을 사용하여 더 큰 스케일의 모델로 ChatGPT를 re-building해 모델 성능을 향상시켰다

# Related Packages

In [ ]:
!pip install chatgpt

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.optim import Adam
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments
from copy import deepcopy
import copy
import logging
import json
from dataclasses import dataclass
# Proximal Policy Optimization (PPO)
from copy import deepcopy
#from chatgpt.models.base import RewardModel
#from chatgpt.models.gpt import GPTActor, GPTCritic
#from chatgpt.trainer import PPOTrainer
#from chatgpt.trainer.strategies import NaiveStrategy
from transformers import AutoTokenizer

In [4]:
model_name = "skt/kogpt2-base-v2"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(
    model_name, bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

In [5]:
print(tokenizer)

GPT2TokenizerFast(name_or_path='skt/kogpt2-base-v2', vocab_size=51200, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '</s>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=True)


# SFT 

In [7]:
from typing import Optional, Dict, Sequence

In [8]:
class SFT_dataset(Dataset):

    def __init__(self, data_path_1_SFT: str, tokenizer: transformers.PreTrainedTokenizer, verbose=False):
        super(SFT_dataset, self).__init__()
        logging.warning("Loading data...")

        pattern_instruction = 'prompt'  # instruction
        pattern_output = 'completion'  # response

        data_path_1_SFT = '/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl'
        with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
            list_data_dict = json.load(json_file)

        PROMPT_DICT = {
            "prompt_input": (
                "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
            )
        }

        prompt_input = PROMPT_DICT["prompt_input"]

        sources = []
        for example in list_data_dict:
            tmp = prompt_input.format_map(example)
            sources.append(tmp)

        targets = []
        for example in list_data_dict:
            targets.append(f"{example[pattern_output]}{tokenizer.eos_token}")
        examples = [s + t for s, t in zip(sources, targets)]

        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # source
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # source + target

        input_ids = examples_tokenized["input_ids"]
        labels = copy.deepcopy(input_ids)
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = -100

        data_dict = dict(input_ids=input_ids, labels=labels)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))


    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                max_length=tokenizer.model_max_length,
                truncation=True,
            )
            for text in strings
        ]
        input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
        input_ids_lens = labels_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
        ]
        return dict(
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )


    def __len__(self):
        return len(self.input_ids)


    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])

In [9]:
@dataclass
class DataCollatorForSupervisedDataset(object): 

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value= -100)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [10]:
train_dataset = SFT_dataset(data_path_1_SFT='./aiffel/KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl', tokenizer=tokenizer)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

print('input : %s'%train_dataset.input_ids[0])
print('output: %s'%train_dataset.labels[0])

input : tensor([  739,   378,   378,   378, 14659, 13394, 37091, 10651,   383, 25841,
         8006, 14914,   375,  7673, 20479,  8091, 22311,  9036, 30902, 13675,
          375,   378,   378,   378, 41951,   454,  9549, 20549,   383,  8142,
         7192, 14914,   382, 37767, 13753,  8263,  7166,   739,  8352,  7659,
         9594, 25585, 13600,  8022,  9378, 11532,  9887, 11218,  9111, 16691,
        10351, 10561,  9128, 20479,  8091,  9065,  9446,  9036, 28420, 26521,
        10163, 26367,  6958,  9030,  9882, 12317, 25882,  9209, 37194, 10351,
         9036, 12168, 10529, 15989,  9719, 15434, 10552, 11188, 13362,  9036,
        15805, 11300, 11846,  9146, 16691,  9181,  7397, 15806, 13480, 11342,
        17596,  9161, 19996,  9025, 25006, 18595,  9966, 12592, 10751, 11814,
         8711,  9046, 12450,  9117,  7377, 12521,     1])
output: tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -10

In [11]:
training_args = TrainingArguments(
    output_dir="aiffel/KoChatGPT/test",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=5,
    prediction_loss_only=True,
    fp16 = True
    )
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

In [12]:
trainer.train()
model.save_pretrained('/aiffel/KoChatGPT/output_1_SFT')

/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,2.984100
1000,2.776800
1500,2.687200


> Evaluate

In [13]:
generator = pipeline('text-generation', model='/aiffel/KoChatGPT/output_1_SFT', tokenizer=tokenizer)

generation_args = dict(   
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=375, # \n   
    max_new_tokens=64,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

In [14]:
PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

In [15]:
list_prompt = ['르세라핌 데뷔연도는?',
               '미세먼지가 얼마나 심할 때 마스크 씁니까?',
               '모두의연구소는 어떤 기업입니까?',
               '디지털 헬스케어는 무엇인가요?']

In [16]:
list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt' : tmp}) for tmp in list_prompt]

list_result = generator(list_prompt, **generation_args)   
for prompt, result in zip(list_prompt, list_result):
    print()
    print((result[0]['generated_text']))

/opt/conda/lib/python3.9/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(



### Instruction(명령어):
르세라핌 데뷔연도는?

### Response(응답):'르세라닉 데뷔연도는 2019년입니다.律, 律, 法, 律 등 다양한 분야에서 활동하고 있습니다.律, 法은 정치, 경제, 사회, 문화 등 다양한 분야에서 활약하는 것을 의미합니다.律은 주로 예술 분야에서 활동하며, 법과 律은 영화나 드라마 등에서 많이 사용됩니다.

### Instruction(명령어):
미세먼지가 얼마나 심할 때 마스크 씁니까?

### Response(응답):'미세먼지 농도는 매우 높기 때문에 미세먼지 농도가 높을 때는 마스크를 착용하지 않는 것이 좋습니다. 또한, 미세먼지는 공기 중에 먼지나 오염물질이 쌓이는 것을 말합니다. 따라서 미세먼지를 줄일 수 있는 방법을 찾아보시는 것은 어떨까요?明筆)明

### Instruction(명령어):
모두의연구소는 어떤 기업입니까?

### Response(응답):'저는 인공지능 어시스턴트이기 때문에 모두의연구소에 대한 정보를 가지고 있지 않습니다. 하지만 일반적으로, 모두의연구소는 다양한 분야에서 활동하고 있습니다. 예를 들어, 의료분야에서는 서울대학교병원과 서울아산병원, 국립암센터 등이 있습니다. 또한, 산업 분야에서는 한국전기안전기술원(KEITI), 한국전력

### Instruction(명령어):
디지털 헬스케어는 무엇인가요?

### Response(응답):'디지털 헬스 (Digital Health) 또는 디지털 헬스 (Medicine Health)은 디지털 헬스(Digital Hells)의 약어입니다. 이 용어는 인공지능(AI) 기술을 활용하여 인간의 건강 상태를 평가하고 진단하는 것을 말합니다. 예를 들어, 건강한 식습관과


> 뜬금없이 out-of-context 한자가 나오기도 하고, 틀린 정보다 알려줍니다. 빔서치를 적용했는데, repeat이 응답으로 나왔네요.

> BLEU Score 

In [19]:
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu

In [17]:
good_response_label = [
    '르세라핌 데뷔연도는 2022년이다.',
    '마스크 착용 기준은 명확하게 있지는 않습니다. 하지만, 일반적으로 미세먼지 농도가 PM10 기준으로 81㎍/㎥ 이상, PM2.5 기준으로 36㎍/㎥ 이상일 때는 마스크를 착용하는 것이 좋다고 권고하고 있습니다.',
    '모두의연구소는 인공지능(AI)과 관련된 교육과 연구를 하는 커뮤니니티 기업입니다',
    '디지털 헬스케어는 의료 서비스 제공과 결과를 개선하기 위해 디지털 기술을 사용하는 것을 가리키는 용어입니다.  세계보건기구(WHO)에 따르면 디지털 의료는 "정보 및 통신 기술(ICT)을 사용하여 의료 서비스를 제공하고, 의료 시스템 관리를 지원하고, 사람들이 건강 정보에 액세스할 수 있도록 하고, 건강한 라이프스타일을 장려하고, 과학적 지식을 생성하는 것"이라고 정의합니다.'
]

In [20]:
for prompt, result, label in zip(list_prompt, list_result, good_response_label):
    q = prompt[prompt.index('\n')+1:prompt.rindex('\n')]
    print('Q.', q )
    g = result[0]['generated_text'][result[0]['generated_text'].rindex('(응답):')+5:]
    print('G.', g )
    print()
    a = label # 정답
    print('A.', label)
    print()
    print("1-Gram BLEU :", sentence_bleu(a.split(), g.split(), weights=(1, 0, 0, 0 )))  
    print("2-Gram BLEU :", sentence_bleu(a.split(), g.split(), weights=(0, 1, 0 ,0 )))  
    print("3-Gram BLEU :", sentence_bleu(a.split(), g.split(), weights=(0, 0, 1 ,0 )))  
    print("4-Gram BLEU :", sentence_bleu(a.split(), g.split(), weights=(0, 0, 0, 1 ))) 
    print('-'*50)


Q. 르세라핌 데뷔연도는?

G. '르세라닉 데뷔연도는 2019년입니다.律, 律, 法, 律 등 다양한 분야에서 활동하고 있습니다.律, 法은 정치, 경제, 사회, 문화 등 다양한 분야에서 활약하는 것을 의미합니다.律은 주로 예술 분야에서 활동하며, 법과 律은 영화나 드라마 등에서 많이 사용됩니다.

A. 르세라핌 데뷔연도는 2022년이다.

1-Gram BLEU : 0
2-Gram BLEU : 0
3-Gram BLEU : 0
4-Gram BLEU : 0
--------------------------------------------------
Q. 미세먼지가 얼마나 심할 때 마스크 씁니까?

G. '미세먼지 농도는 매우 높기 때문에 미세먼지 농도가 높을 때는 마스크를 착용하지 않는 것이 좋습니다. 또한, 미세먼지는 공기 중에 먼지나 오염물질이 쌓이는 것을 말합니다. 따라서 미세먼지를 줄일 수 있는 방법을 찾아보시는 것은 어떨까요?明筆)明

A. 마스크 착용 기준은 명확하게 있지는 않습니다. 하지만, 일반적으로 미세먼지 농도가 PM10 기준으로 81㎍/㎥ 이상, PM2.5 기준으로 36㎍/㎥ 이상일 때는 마스크를 착용하는 것이 좋다고 권고하고 있습니다.

1-Gram BLEU : 0
2-Gram BLEU : 0
3-Gram BLEU : 0
4-Gram BLEU : 0
--------------------------------------------------
Q. 모두의연구소는 어떤 기업입니까?

G. '저는 인공지능 어시스턴트이기 때문에 모두의연구소에 대한 정보를 가지고 있지 않습니다. 하지만 일반적으로, 모두의연구소는 다양한 분야에서 활동하고 있습니다. 예를 들어, 의료분야에서는 서울대학교병원과 서울아산병원, 국립암센터 등이 있습니다. 또한, 산업 분야에서는 한국전기안전기술원(KEITI), 한국전력

A. 모두의연구소는 인공지능(AI)과 관련된 교육과 연구를 하는 커뮤니니티 기업입니다

1-Gram BLEU : 0
2-Gram BLEU : 0
3-Gram BLE

/opt/conda/lib/python3.9/site-packages/nltk/translate/bleu_score.py:515: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/opt/conda/lib/python3.9/site-packages/nltk/translate/bleu_score.py:515: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/opt/conda/lib/python3.9/site-packages/nltk/translate/bleu_score.py:515: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

In [21]:
torch.cuda.empty_cache()

# RM (Reward Model)

In [22]:
!pip install chatgpt

     |████████████████████████████████| 36.9 MB 12.4 MB/s            


In [23]:
!pip install "$HOME/aiffel/KoChatGPT/colossalai_ChatGPT_230319/"

Processing /aiffel/aiffel/KoChatGPT/colossalai_ChatGPT_230319
  Preparing metadata (setup.py) ... done


  Created wheel for chatgpt: filename=chatgpt-0.1.0-py3-none-any.whl size=46664 sha256=71dcbbdc77a96e8710e353363f83c0341d6e07f83977ed7898228904aacd85b9
  Stored in directory: /tmp/pip-ephem-wheel-cache-r92lbwhs/wheels/79/25/c3/338e0c56a2253a8ea6c41e8692f6eb2409a3898c63b234b103
Successfully built chatgpt
  Attempting uninstall: chatgpt
    Found existing installation: chatgpt 2.2212.0
    Uninstalling chatgpt-2.2212.0:
      Successfully uninstalled chatgpt-2.2212.0


In [24]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import json
from typing import Optional

import torch
import torch.nn as nn
from torch.optim import Adam
from chatgpt.dataset import RewardDataset
from chatgpt.models.base import RewardModel
from chatgpt.trainer import RewardModelTrainer
from chatgpt.trainer.strategies import NaiveStrategy
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel, AutoConfig
from transformers.models.gpt2.configuration_gpt2 import GPT2Config
from transformers.models.gpt2.modeling_gpt2 import GPT2Model
import loralib as lora

In [25]:
class GPTRM_custom(RewardModel):

    def __init__(self,
                 pretrained: Optional[str] = None,
                 config: Optional[GPT2Config] = None,
                 checkpoint: bool = False,
                 lora_rank: int = 0,
                 lora_train_bias: str = 'none',
                 tokenizer=None) -> None:
        if pretrained is not None:
            model = GPT2Model.from_pretrained(pretrained)
            model.resize_token_embeddings(len(tokenizer))
        elif config is not None:
            model = GPT2Model(config)
        else:
            model = GPT2Model(GPT2Config())
        if checkpoint:
            model.gradient_checkpointing_enable()

        value_head = nn.Linear(model.config.n_embd, 1)
        super().__init__(model, value_head, lora_rank, lora_train_bias)

        if pretrained is not None:
            self.model = model
            self.pretrained = pretrained


    def save_pretrained(self, dir):
        if self.pretrained is not None:
            self.model.save_pretrained(dir)

In [26]:
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(
    model_name, bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

In [27]:
with NaiveStrategy().model_init_context():
        model = GPTRM_custom(pretrained=model_name, 
                             lora_rank=0, tokenizer=tokenizer).cuda()

Some weights of the model checkpoint at skt/kogpt2-base-v2 were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:
with open('/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_2_RM.jsonl', "r", 
          encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

total_data_ranking2chosen = []
for tmp in list_data_dict:
    one_data_ranking2chosen = []

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][1]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_1']
    else:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][1] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_1']
    one_data_ranking2chosen.append(data)



    total_data_ranking2chosen.extend(one_data_ranking2chosen)

In [29]:
print('before data num: %d'%(len(list_data_dict)))
print('after  data num: %d'%(len(total_data_ranking2chosen)))
print('data example: \n%s'%total_data_ranking2chosen[45])

before data num: 10220
after  data num: 30660
data example: 
{'prompt': '애플은 리사를 어떻게 처리했어', 'chosen': '애플이 누구인지 명확히 알 수 없어서, 리사가 누구인지와 어떤 상황에서 처리되었는지에 대한 추가적인 정보가 필요합니다. 따라서, 보다 정확한 답변을 제공할 수 없습니다.', 'rejected': '애플은 리사를 위해 고객 서비스 부서에서 고객 다양한 컴퓨터 관련 문제에 대해 응답하는 데 필요한 모든 지원을 제공했습니다. 사용자가 하드웨어 문제를 경험할 때, 전문가들은 필요한 수리(수리, 추가 부품 제공, 소프트웨어 업그레이드 등)을 제공해 드릴 수 있습니다. 또한, 사용자가 사용 방법 문제나 기타 문제를 경험할 때, 대화 상대로 사용자를 지원할 수 있는 전문 고객 서비스 직원들이 사용자에게 상담하고 도움을 주는 데 도움이 될 수 있는 정보를 제공합니다. 또한, 인터넷에서 제공되는 정보를 통해 문제를 해결하거나 고객 서비스 웹 사이트를 통해 자신의 문제를 진단할 수 있도록 하는 등 다양한 방법으로 리사를 처리해 왔습니다.'}


In [30]:
import random
random.seed(231208)
random.shuffle(total_data_ranking2chosen)
print(total_data_ranking2chosen[45])

{'prompt': '이 카드 무이자 할부 되죠?', 'chosen': '죄송하지만 저는 이직하고부터 은행에서 일하고 있지 않습니다. 하지만 대개 카드 무이자 할부는 해당 카드 발급사의 조건에 따라 달라지기 때문에, 해당 카드 발급사에 문의하시거나 해당 카드의 웹사이트를 참고하시는 것이 좋을 것 같습니다.', 'rejected': '이상활돌들 개인사이에서도 국가미남 국가미남의 국가미남을 국가미남의 국가미남의 국가미남의 국가미남의 국가미남의 국가미남의 국가미남의 국가미남의 국가미남의 국가미남의 국가미남의 국가미남의 국가미남의 국가미남의 국가미'}


In [31]:
train_data = total_data_ranking2chosen[:1000] 
eval_data = total_data_ranking2chosen[1000:1200]

print(len(train_data))
print(len(eval_data))

train_dataset = RewardDataset(train_data, tokenizer, 512)
eval_dataset = RewardDataset(eval_data, tokenizer, 512)

1000
200


100%|██████████| 200/200 [00:00<00:00, 1364.86it/s]


In [32]:
idx = 1
print('#'*70)
print('## prompt ##')
print(train_data[idx]['prompt'])
print('#'*70)
print('## chosen ##')
print(train_data[idx]['chosen'])
print('#'*70)
print('## rejected ##')
print(train_data[idx]['rejected'])

######################################################################
## prompt ##
바이오하자드 엄브렐러 크로니클즈는 몇 인칭 게임인가요?
######################################################################
## chosen ##
바이오하자드 엄브렐러 크로니클즈는 세 인칭 게임입니다.
######################################################################
## rejected ##
이상화를 한 개인으로 크로니클즈는 개인으로 크로니클즈를 한 개인으로 크로니클즈를 한 개인으로 크로니클즈를 한 개인으로 크로니클즈를 한 개인으로 크로니클즈를 한 개인으로 크로니클즈를 한 개인으로 크로니클즈를 한 개인으로 크로니


In [33]:
trainer = RewardModelTrainer(model=model,
                             strategy=NaiveStrategy(),
                             optim=Adam(model.parameters(), lr=5e-5),
                             train_dataset=train_dataset,
                             eval_dataset=eval_dataset,
                             batch_size=4,
                             max_epochs=1)

In [34]:
trainer.fit(use_lora=0)

model.save_pretrained('/aiffel/KoChatGPT/output_2_RM')

Train step of epoch 0:  19%|█▉        | 47/250 [00:39<02:51,  1.18it/s, loss=0.656]


Train step of epoch 0:  38%|███▊      | 94/250 [01:19<02:17,  1.14it/s, loss=0.635]


Train step of epoch 0:  56%|█████▌    | 140/250 [02:00<01:35,  1.15it/s, loss=0.875]


Train step of epoch 0:  75%|███████▍  | 187/250 [02:40<00:54,  1.15it/s, loss=0.429]


Train step of epoch 0:  93%|█████████▎| 233/250 [03:20<00:14,  1.15it/s, loss=0.701]


Train epoch: 100%|██████████| 1/1 [03:49<00:00, 229.91s/it]


In [35]:
def inference_RM(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    output = model(input_ids)
    output_reward = output.cpu().detach().numpy()[0]

    print('input: %s\nreward score: %.1f'%(input_text, output_reward))

    return output_reward

input_text = '인공지능은 똥멍청이 입니다'
output_reward = inference_RM(input_text=input_text)

input: 인공지능은 똥멍청이 입니다
reward score: -0.7


In [36]:
input_text = '인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다.'
output_reward = inference_RM(input_text=input_text)

input: 인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다.
reward score: -0.5


In [37]:
input_text = "인공지능은 일반적으로 인간의 지능이 필요하거나 인간이 분석할 수 있는 것보다 규모가 큰 데이터를 포함하는 방식으로 추론, 학습 및 행동할 수 있는 컴퓨터 및 기계를 구축하는 것과 관련된 과학 분야입니다. AI는 컴퓨터 공학, 데이터 분석 및 통계, 하드웨어 및 소프트웨어 엔지니어링, 언어학, 신경 과학은 물론 철학과 심리학을 포함하여 여러 학문을 포괄하는 광범위한 분야입니다. 비즈니스의 운영 수준에서 AI는 주로 머신러닝과 딥 러닝을 기반으로 하는 기술 모음으로, 데이터 분석, 예상 및 예측, 객체 분류, 자연어 처리, 추천, 지능형 데이터 가져오기 등을 수행할 수 있습니다."

output_reward = inference_RM(input_text=input_text)

input: 인공지능은 일반적으로 인간의 지능이 필요하거나 인간이 분석할 수 있는 것보다 규모가 큰 데이터를 포함하는 방식으로 추론, 학습 및 행동할 수 있는 컴퓨터 및 기계를 구축하는 것과 관련된 과학 분야입니다. AI는 컴퓨터 공학, 데이터 분석 및 통계, 하드웨어 및 소프트웨어 엔지니어링, 언어학, 신경 과학은 물론 철학과 심리학을 포함하여 여러 학문을 포괄하는 광범위한 분야입니다. 비즈니스의 운영 수준에서 AI는 주로 머신러닝과 딥 러닝을 기반으로 하는 기술 모음으로, 데이터 분석, 예상 및 예측, 객체 분류, 자연어 처리, 추천, 지능형 데이터 가져오기 등을 수행할 수 있습니다.
reward score: -0.1


In [38]:
torch.cuda.empty_cache()

# PPO

In [39]:
from copy import deepcopy

import json

import torch
from torch.optim import Adam
from chatgpt.models.base import RewardModel
from chatgpt.models.gpt import GPTActor, GPTCritic
from chatgpt.trainer import PPOTrainer
from chatgpt.trainer.strategies import NaiveStrategy
from transformers import AutoTokenizer

In [47]:
with NaiveStrategy().model_init_context():
    actor = GPTActor(pretrained='/aiffel/KoChatGPT/output_1_SFT', lora_rank=0).to(torch.cuda.current_device())
    critic = GPTCritic(pretrained='/aiffel/KoChatGPT/output_2_RM', lora_rank=0).to(torch.cuda.current_device())

    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right", 
        model_max_length=512
    )

    initial_model = deepcopy(actor)
    reward_model = RewardModel(deepcopy(critic.model), deepcopy(critic.value_head)).to(torch.cuda.current_device())

In [49]:
actor_optim = Adam(actor.parameters(), lr=5e-6)
critic_optim = Adam(critic.parameters(), lr=5e-6)

In [50]:
(actor, actor_optim), (critic, critic_optim), reward_model, initial_model = NaiveStrategy().prepare(
    (actor, actor_optim), (critic, critic_optim), reward_model, initial_model)

In [51]:
with open('/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)
    list_prompt = [tmp['prompt'] for tmp in list_data_dict]

def tokenize_fn(texts):
    batch = tokenizer(texts, return_tensors='pt', max_length=96, padding=True, truncation=True)
    return {k: v.cuda() for k, v in batch.items()}

In [52]:
print(tokenize_fn('It takes something more than intelligence to act intelligently.'))

{'input_ids': tensor([[47311, 10448, 19008,  9792, 11780, 11308, 30190, 10929, 11849, 21663,
         44389,  9574, 13799,   458, 14308, 12778, 22469, 20938, 44696,   458,
         13799,   458, 14308, 12778, 11756, 18944,   389]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]], device='cuda:0')}


In [53]:
len(list_prompt)

12000

In [54]:
trainer = PPOTrainer(NaiveStrategy(),
                     actor,
                     critic,
                     reward_model,
                     initial_model,
                     actor_optim,
                     critic_optim,
                     max_epochs=1,  
                     train_batch_size=8, 
                     tokenizer=tokenize_fn,
                     max_length=128,
                     do_sample=True,
                     temperature=1.0,
                     top_k=50,
                     pad_token_id=tokenizer.pad_token_id,
                     eos_token_id=tokenizer.eos_token_id)

In [55]:
trainer.fit(list_prompt, 
            num_episodes=10,  
            max_timesteps=3,
            update_timesteps=3)

model.save_pretrained('aiffel/KoChatGPT/output_3_PPO')

Episode [10/10]: 100%|██████████| 3/3 [00:17<00:00,  5.84s/it]


RLHF가 적용된 koGPT-2의 생성능력을 확인해볼까요?

In [56]:
def generation(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    outputs = actor.generate(input_ids,
                             max_length=250,
                             do_sample=True,
                             top_k=50,
                             top_p=0.95,
                             num_return_sequences=1)
    output = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)[0]
    print()
    print(output)
    return output

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = ['르세라핌 데뷔연도는?',
               '미세먼지가 얼마나 심할 때 마스크 씁니까?',
               '모두의연구소는 어떤 기업입니까?',
               '디지털 헬스케어는 무엇인가요?']

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt': tmp}) for tmp in list_prompt]

for input_text in list_prompt:
    output = generation(input_text)


### Instruction(명령어):
르세라핌 데뷔연도는?

### Response(응답):'르세라핌 데뷔연도는 2018년입니다. Q, C, E, J, EN, J는 모두 르세라핌 멤버의 이름이며, 이후 K는 2016년에도 이 연애를 하고 있습니다. EN, J는 르세라핌이라는 솔로 가수의 이름으로 불리며, 이후 그 해에 ONA를 상대로 승부에서 승리하였습니다. J는 SKT, K는 SS 멤버로써 KSM 무대에서 다양한 공연을 즐기며 KRC를 이끌어내며 우승을 차지하는데 큰 기여를 하였습니다. J는 KT와 KSM의 데뷔연애를 주도하며, 다양한 무대에서 KT를 빛나게 만든 멤버 중 한 명입니다. J는 KSM의 대표적인 가수로써, SSM의 대표 멤버 중 한 명입니다. KSM의 리더로, C, EN, J은 KSM의 홍보위원으로 참여하여 KSM의 이미지를 확립시키는데 큰 역할을 하였습니다. EN, J는 KSM의 공식 멤버, KSM의 공식 커머스에서 KSM의 브랜드

### Instruction(명령어):
미세먼지가 얼마나 심할 때 마스크 씁니까?

### Response(응답):'미세먼지가 심하다고 느끼기 시작하면 알레르기 반응이 매우 급격하게 올 수 있고, 그 결과가 매우 심각한 알레르기 반응을 발생시킬 수 있습니다. 미세먼지는 대개 호흡기와 기관에 큰 영향을 미치기 때문에 실내에서 오랫동안 실내 생활을 영위하는 경우가 많습니다. 그러나 미세먼지를 제거하는 최선의 방법은 생활 습관을 바꿀 필요가 있는 것을 알려주고, 호흡기와 기관 내부의 청결을 유지하고 미세먼지와 알레르기 유발 물질을 효과적으로 막는 방법 중 하나일 수 있습니다. 또한, 미세먼지 제거 효과도 피부나 호흡기 보호막 역할을 할 수 있으므로, 건강 상태를 수시로 체크하는 것이 좋습니다.現\n  \n하지만 이러한 노력에도 불구하고 알레르기 반응이 발생하면 증상이 심할 수 있으며, 각별한 관리가 필요합니다. 적극적인 치료를 위해서는 먼저 환자의 상태를 정확하게 파악하고 그에 맞는 적절한 치료를 받을 수 있는 전

> 회고록 <br> 
> chatGPT가 이러한 방식으로 간략히 볼 수 있어서 너무 좋았다. 또한 RLHF를 적용하니, 좀 더 context에 맞는 것들을 생성하고 있었다. 
하지만 여전히 많이 부족했다. 좀 더 정확히 하기 위해서 여러가지 파라미터들을 바꿔보고, 모델 세부를 바꿔보는 그러한 노력들을 할것 같습니다.